In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd

from src.paths import PREPROCESSED_DATA_DIR
from src.features import aggregate_columns

In [3]:
df_weather = pd.read_pickle(PREPROCESSED_DATA_DIR / 'clean_weather.pkl')
print(f'Columns: {df_weather.columns.to_list()}')

Columns: ['Date', 'Tmax_1', 'Tmin_1', 'Tavg_1', 'DewPoint_1', 'WetBulb_1', 'PrecipTotal_1', 'Tmax_2', 'Tmin_2', 'Tavg_2', 'DewPoint_2', 'WetBulb_2', 'PrecipTotal_2']


In [4]:
df_weather.set_index('Date', inplace=True)

In [6]:
df_agg = aggregate_columns(df_weather, lag_range=(1, 15, 2), window_range=(3,10,2), agg_func='mean')

In [8]:
df_agg.to_pickle(PREPROCESSED_DATA_DIR / 'aggregated_weather.pkl')